In [174]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from math import *
import scipy.stats
from pandas import *
#import scipy.integrate as integrate
from scipy.integrate import quad
import seaborn as sns
import scipy as s
import warnings
warnings.filterwarnings("ignore")
from mpl_toolkits import mplot3d

import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.graph_objects as go
from scipy import linalg 


In [175]:
def jump_distribution_full(x_value,mu,sigma_jump,step_x):
    lower = x_value - step_x/2.0
    upper = x_value + step_x/2.0
    #print(lower,upper)
    def normal_distribution_function(x):
        value = scipy.stats.norm.pdf(x,mu,sigma_jump)
        return value
    res, err = quad(normal_distribution_function, lower, upper)
    return res  #output is a vector of the average densities around the regions of each value in the vector x


In [178]:
x0 = -10.0
xn = 10.0
xsteps = 1000
dx = (xn-x0)/xsteps
x = np.arange(x0, xn+dx, dx) #last x value is x = 99 
#print(x,len(x),dx)

In [179]:
t0 = 0.0
tn = 1.0
tsteps = 100
dt = (tn-t0)/tsteps
t = np.arange(t0, tn+dt, dt) #last time value is t = 999.9
r = dt / (dx**2) # ensure r < 1
print(dt)
#print(t,r,len(t),tsteps,dt)

0.01


In [180]:
def a_coeff(x_value, sigma, kappa, theta, jump_rate):
    coefficient = jump_rate + (sigma**2)/(dx**2)
    return coefficient

def b_coeff(x_value, sigma, kappa, theta, jump_rate):
    coefficient_2 = 0.5*(sigma**2)*(1/dx**2) + 0.5*(1/dx)*kappa*(theta-x_value)
    return coefficient_2

def c_coeff(x_value, sigma, kappa, theta, jump_rate):
    coefficient_3 = 0.5*(sigma**2)*(1/dx**2) - 0.5*(1/dx)*kappa*(theta-x_value)
    return coefficient_3   

In [181]:
def M_matrix(x,sigma,kappa, theta, jump_rate):
    M = np.zeros(shape=(len(x),len(x)))
    for i in range(0,len(x)):
        if x[i] <= 0 or x[i] >= 1:
            M[i,i] = 1.0
        else:
            a_coefficient = a_coeff(x[i], sigma, kappa, theta, jump_rate)
            b_coefficient = b_coeff(x[i], sigma, kappa, theta, jump_rate)
            c_coefficient = c_coeff(x[i], sigma, kappa, theta, jump_rate)
            if a_coefficient < 0 or b_coefficient < 0 or c_coefficient <0:
                print('Error')
            M[i,(i-1):(i+2)] = [-dt*c_coefficient, 1+dt*a_coefficient, -dt*b_coefficient]

    return M

In [182]:
##accept_x = np.where((x>=0) & (x<=1))[0]
#accept_x

In [183]:
#M_1 = M_matrix(x,0.1,0.7,1.0,1.0)
#M_inv = np.linalg.inv(M_1)
#DataFrame(M_1[50:62,50:62])

In [184]:
def N_matrix(x, jump_rate, N, mu, sigma_jump, step_x):
    N_m = np.zeros(shape=(len(x),len(x)))
    positions = np.arange(int(-N/2+1),int(N/2),1)
    for i in range(0,(len(x))):
        if x[i] > 0.0 and x[i] < 1.0:
            for j in positions: 
                x_jump = step_x*j
                integral = jump_distribution_full(x_jump, mu, sigma_jump, step_x)
                if (i+j) >= 0 and (i+j) <= (len(x)-1):
                    #print(j)
                    N_m[i,i+j] = jump_rate*dt*integral
            N_m[i,i] = N_m[i,i]+1
    return N_m

In [185]:
#N_try = N_matrix(x, 1.0,4,0.0,0.1,dx)
#DataFrame(N_try[50:62,50:62])

In [186]:
#N_try = N_matrix(x, 1.0,4,0.0,0.1,dx)
#DataFrame(N_try[0:10, 0:10])

In [187]:
#def IC_matrix(x, sigma, kappa, theta, jump_rate, mu, sigma_jump, step_x, phi_0, phi_N):
#    b = np.zeros(len(x)-2)
#    B = np.zeros(len(x)-2)
#    b_coefficient = b_coeff(x[1], sigma, kappa, theta, jump_rate)
#    c_coefficient = c_coeff(x[len(x)-2], sigma, kappa, theta, jump_rate)
#    b[0] = c_coefficient*dt*phi_0
#    b[len(x)-3] = b_coefficient*dt*phi_N
    #print(b)
#    jump_0 = jump_distribution_full(-1*dx, mu, sigma_jump, step_x)
#    jump_N = jump_distribution_full(1*dx, mu, sigma_jump, step_x)
#    B[0] = jump_rate*dt*jump_0
#    B[len(x)-3] = jump_rate*dt*jump_N
    
#    IC = b+B
#    return IC

In [188]:
#IC_matrix(x,1,1,1,1,0,1,dx,0,1)

In [189]:
b = np.zeros(len(x))
b[x>=1.0] = 1.0

In [190]:
phi_matrix = np.zeros(shape=(len(x), len(t)))
phi_matrix[:,0] = 1.0 * (x>0.0)
phi_matrix[len(x)-1,:] = 1.0
#DataFrame(phi_matrix)

In [221]:
sigma_choice = 0.5
kappa_choice = 0.5
theta_choice = 0.3
jump_rate_choice = 1.0
N_choice = 4
mu_choice = 0.0
sigma_jump_choice = 0.5

M = M_matrix(x,sigma = sigma_choice,kappa = kappa_choice, theta = theta_choice, jump_rate = jump_rate_choice)
#print(DataFrame(M[13:28,13:28]))
N_mat = N_matrix(x, jump_rate = jump_rate_choice, N = N_choice, mu = mu_choice, sigma_jump = sigma_jump_choice, step_x=dx)
#print(DataFrame(N_mat[13:28,13:28]))
#print(b[0])

if sigma_choice**2 >= np.max(np.abs(dx*kappa_choice*(theta_choice-x))):
    for time in range(1, len(t)): 
        phi_matrix[:,time] = np.linalg.inv(M).dot(N_mat.dot(phi_matrix[:,time-1]) + b)
        #print(DataFrame(np.dot(np.linalg.inv(M), M)))
        #phi_matrix[:,time] = linalg.solve(M, N_mat.dot(phi_matrix[:,time-1]) + b )
        
else:
    print('Stability conditions may not be satisfied')
    

In [222]:
accept_x = np.where((x>=0) & (x<=1))[0]
accept_x

array([501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513,
       514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526,
       527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539,
       540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550], dtype=int64)

In [223]:
phi_matrix[26,] 

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [63]:
for i in range(0,len(phi_matrix)-1):
    print(np.min(phi_matrix[i+1,:] - phi_matrix[i,:]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [64]:
for i in range(0,len(t)-1):
    print(np.max(phi_matrix[:,i+1] - phi_matrix[:,i]))

1.360308049807529e-17
0.0
1.7291167724525608e-18
1.6170948666546702e-19
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [224]:
def survival(space,time):
    return phi_matrix[space, time]


In [225]:
X = accept_x #np.arange(50,65,1)
T = np.arange(0,len(t),1)
X, T = np.meshgrid(X, T)
Phi = survival(X,T)

data=go.Surface(z=Phi, x=x0+dx*X, y=t0+dt*T)

layout = go.Layout(scene = dict(
                    xaxis_title='Initial Position',
                    yaxis_title='Maturity',
                    zaxis_title='Survival Probability'))

fig = go.Figure(data=[data], layout=layout)
pyoff.plot(fig)

'temp-plot.html'

In [226]:
surv = DataFrame(phi_matrix)

In [227]:
surv['initial_pos'] = x #x0+dx*X[1]

In [228]:
times= DataFrame(t)
time= times[0]

In [229]:
surv.append(time)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,initial_pos
0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-10.00
1,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.98
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.96
3,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.94
4,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-9.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,9.94
998,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,9.96
999,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,9.98
1000,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,10.00


In [230]:
x[550]

0.9999999999997655

In [231]:
y = np.where((x >= 0.0) & (x<= 1.02))
surv_df_non_zero = surv.iloc[y[0],]
surv_df_non_zero

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,initial_pos
501,1.0,0.431494,0.275873,0.212309,0.177789,0.155551,0.139684,0.127600,0.117977,0.110060,...,0.016466,0.016245,0.016028,0.015816,0.015608,0.015405,0.015205,0.015010,0.014819,0.02
502,1.0,0.674788,0.497204,0.400433,0.341248,0.301084,0.271672,0.248933,0.230649,0.215507,...,0.032546,0.032109,0.031681,0.031262,0.030852,0.030450,0.030057,0.029672,0.029294,0.04
503,1.0,0.812063,0.659215,0.555233,0.483619,0.431804,0.392443,0.361315,0.335905,0.314636,...,0.048251,0.047605,0.046971,0.046352,0.045745,0.045150,0.044568,0.043998,0.043440,0.06
504,1.0,0.889580,0.771667,0.675792,0.602034,0.544912,0.499621,0.462780,0.432115,0.406085,...,0.063594,0.062744,0.061912,0.061098,0.060300,0.059519,0.058754,0.058005,0.057271,0.08
505,1.0,0.933390,0.847070,0.765865,0.696795,0.639565,0.592065,0.552181,0.518221,0.488904,...,0.078586,0.077540,0.076515,0.075512,0.074530,0.073569,0.072627,0.071705,0.070801,0.10
506,1.0,0.958169,0.896421,0.831033,0.770221,0.716474,0.669745,0.629166,0.593741,0.562575,...,0.093238,0.092002,0.090793,0.089608,0.088448,0.087312,0.086200,0.085110,0.084043,0.12
507,1.0,0.972195,0.928151,0.876999,0.825599,0.777367,0.733488,0.694055,0.658717,0.626990,...,0.107561,0.106144,0.104756,0.103397,0.102066,0.100763,0.099486,0.098236,0.097011,0.14
508,1.0,0.980142,0.948276,0.908763,0.866423,0.824496,0.784684,0.747677,0.713620,0.682391,...,0.121567,0.119976,0.118417,0.116891,0.115396,0.113932,0.112499,0.111094,0.109718,0.16
509,1.0,0.984648,0.960906,0.930349,0.895937,0.860251,0.825012,0.791186,0.759232,0.729300,...,0.135269,0.133511,0.131789,0.130103,0.128452,0.126835,0.125250,0.123698,0.122178,0.18
510,1.0,0.987204,0.968764,0.944813,0.916921,0.886903,0.856228,0.825901,0.796529,0.768437,...,0.148677,0.146761,0.144885,0.143047,0.141246,0.139483,0.137755,0.136063,0.134405,0.20


In [232]:
accept_t = [0,10,20,30,40,50,60,70,80,90,100]

In [233]:
times = t[accept_t[1:]]
times

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [236]:
output=surv_df_non_zero.iloc[:,accept_t[1:]]
initial_pos = surv['initial_pos'][(surv['initial_pos'] >= 0.0) & (surv['initial_pos'] <= 1.02)]
output['initial_pos'] = initial_pos
survival_values = round(output,4)
survival_values = survival_values.set_index('initial_pos')
#survival_values.columns = t[accept_t]
#output
survival_values

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.1034,0.0669,0.0501,0.0396,0.0324,0.0270,0.0228,0.0195,0.0169,0.0148
0.04,0.2027,0.1318,0.0988,0.0782,0.0639,0.0532,0.0450,0.0386,0.0334,0.0293
0.06,0.2965,0.1944,0.1460,0.1157,0.0946,0.0789,0.0667,0.0572,0.0496,0.0434
0.08,0.3836,0.2542,0.1915,0.1521,0.1244,0.1038,0.0879,0.0754,0.0653,0.0573
0.10,0.4633,0.3110,0.2352,0.1872,0.1533,0.1280,0.1085,0.0931,0.0807,0.0708
0.12,0.5349,0.3646,0.2771,0.2210,0.1813,0.1515,0.1285,0.1103,0.0958,0.0840
0.14,0.5984,0.4148,0.3170,0.2536,0.2083,0.1743,0.1480,0.1272,0.1105,0.0970
0.16,0.6538,0.4615,0.3549,0.2847,0.2344,0.1964,0.1669,0.1436,0.1249,0.1097
0.18,0.7014,0.5046,0.3907,0.3145,0.2595,0.2178,0.1853,0.1596,0.1389,0.1222


In [237]:
pd = 1-survival_values
pd

,10,20,30,40,50,60,70,80,90,100
initial_pos,,,,,,,,,,
0.02,0.8966,0.9331,0.9499,0.9604,0.9676,0.9730,0.9772,0.9805,0.9831,0.9852
0.04,0.7973,0.8682,0.9012,0.9218,0.9361,0.9468,0.9550,0.9614,0.9666,0.9707
0.06,0.7035,0.8056,0.8540,0.8843,0.9054,0.9211,0.9333,0.9428,0.9504,0.9566
0.08,0.6164,0.7458,0.8085,0.8479,0.8756,0.8962,0.9121,0.9246,0.9347,0.9427
0.10,0.5367,0.6890,0.7648,0.8128,0.8467,0.8720,0.8915,0.9069,0.9193,0.9292
0.12,0.4651,0.6354,0.7229,0.7790,0.8187,0.8485,0.8715,0.8897,0.9042,0.9160
0.14,0.4016,0.5852,0.6830,0.7464,0.7917,0.8257,0.8520,0.8728,0.8895,0.9030
0.16,0.3462,0.5385,0.6451,0.7153,0.7656,0.8036,0.8331,0.8564,0.8751,0.8903
0.18,0.2986,0.4954,0.6093,0.6855,0.7405,0.7822,0.8147,0.8404,0.8611,0.8778


In [100]:
x = np.array([1.00, 0.5219,0.2464,0.1347,0.1007, 0.0924, 0.0905, 0.0891, 0.0845, 0.0678, 0.00])

In [101]:
x*0.75

array([0.75    , 0.391425, 0.1848  , 0.101025, 0.075525, 0.0693  ,
       0.067875, 0.066825, 0.063375, 0.05085 , 0.      ])

In [104]:
x = np.array([0.2464,0.4346,0.5497,0.6287,0.6869,0.7313,0.7661,0.7937,0.8159,0.8338])
y = np.array([0,0.2464,0.4346,0.5497,0.6287,0.6869,0.7313,0.7661,0.7937,0.8159])
x-y

array([0.2464, 0.1882, 0.1151, 0.079 , 0.0582, 0.0444, 0.0348, 0.0276,
       0.0222, 0.0179])

In [106]:
pos = np.array([100, 90,80,70,60,50,40,30,20,10])

In [109]:
pos*(x-y)*0.75

array([18.48   , 12.7035 ,  6.906  ,  4.1475 ,  2.619  ,  1.665  ,
        1.044  ,  0.621  ,  0.333  ,  0.13425])

In [110]:
sum(pos*(x-y)*0.75)

48.65325

In [111]:
(x-y)*0.75

array([0.1848  , 0.14115 , 0.086325, 0.05925 , 0.04365 , 0.0333  ,
       0.0261  , 0.0207  , 0.01665 , 0.013425])

In [121]:
phi_cd = np.array([1.0,0.9095,0.8250,0.7457,0.6743,0.6126,0.5604,0.5167,0.4805,0.4505,0.4257])

In [140]:
def CDS_numerator(r,dt, Phi):
    Phi_without_last = Phi[0:(len(Phi)-1)]
    Phi_without_0 = Phi[1:]
    t = np.arange(dt, 1+dt, dt)
    #print(t)
    discount = np.exp(-r*t)
    surv = Phi_without_last - Phi_without_0
    #print(surv)
    num_vector = discount*surv
    return(sum(num_vector))

In [141]:
def CDS_denominator(r,dt, Phi):
    Phi_without_last = Phi[0:(len(Phi)-1)]
    Phi_without_0 = Phi[1:]
    t = np.arange(dt, 1+dt, dt)
    discount = np.exp(-r*t)
    surv = Phi_without_0 + Phi_without_last
    num_vector = discount*surv
    return(sum(num_vector))

In [142]:
def CDS_spread(r,dt,Phi,R):
    price = (1-R)*CDS_numerator(r,dt,Phi) / 0.5*dt*CDS_denominator(r,dt,Phi)
    return(price)

In [144]:
CDS_spread(0.1, 0.1, phi_cd,0.5)

0.6805317176128249